In [23]:
#%pip install ujson

In [7]:
#!pip install psycopg2

import psycopg2
import time
import boto3
import copy
import gc
import pandas as pd
import json
import ujson
import os

In [3]:
db_connection = psycopg2.connect(
            user='postgres', password='Data2023',
            host='mads-capstone.cmohac77hep9.eu-north-1.rds.amazonaws.com', port=5432,
            database="mads"
        )

db_connection.autocommit = True

cursor = db_connection.cursor()

In [248]:
#cursor.execute("DROP TABLE submissions;")

In [238]:
query = '''CREATE TABLE submissions (  
                                id VARCHAR(16),
                                subreddit_id VARCHAR(24),
                                subreddit text,
                                author VARCHAR(50),
                                created_utc INT,
                                permalink text,
                                title text,
                                selftext text,
                                num_comments INT,
                                score INT)'''


cursor.execute(query)

In [239]:
import pandas as pd

select_query = "SELECT count(*) FROM submissions"

cursor.execute(select_query)

cursor.fetchall()

[(0,)]

In [4]:
def parse_record(row: str)-> dict:
    try:
        row_data = row.decode(encoding="utf-8", errors="replace").\
                                            replace("null", '""').\
                                            replace("&lt;", "<").\
                                            replace("&gt;", ">").\
                                            replace("&amp;#39;", "").\
                                            replace("&amp;", "").\
                                            replace("false", "False").\
                                            replace("true", "True")
                                            #replace(",", ";")
        #print(row_data)
        row_dict = eval(row_data)
        row_dict["selftext"] = row_dict["selftext"].replace(",", ";").replace("\n", " ").replace("\r", "").replace("\t", "").replace("\\", "") if row_dict["selftext"] else ""
        row_dict["title"] = row_dict["title"].replace(",", ";").replace("\n", " ").replace("\r", "").replace("\t", "").replace("\\", "") if row_dict["title"] else ""
        return row_dict
    except SyntaxError as e: ## Ignore syntax error records
        #print(e)
        pass
        #print(row_data)
    except UnicodeEncodeError as e: ## Ignore Decode error records
        #print(e)
        pass
    except Exception as e: ## Ignore all other errors for continuous processing.
        pass
    return {}

def save_dataframe(records):
    df = pd.DataFrame(records)
    #print(df.columns)
    df = df[selected_columns]
    df.to_csv("submissions.csv", index=False, header=False, encoding="utf-8", errors="replace") 

In [5]:
bucket = 'mads-capstone-2023' 
data_key = 'wallstreetbets_submissions.json' 

selected_columns = ["id", "subreddit_id", "subreddit", "author", "created_utc", "permalink", "title", "selftext", "num_comments", "score",]

s3_client = boto3.client('s3')
obj = s3_client.get_object(Bucket=bucket, Key=data_key)

cnt = 0

for i, row in enumerate(obj["Body"].iter_lines()):
    cnt +=1
    if (i%1_000_000)==0:
        print(f"{i}")
        
cnt

In [244]:
total_records = 2218243

In [233]:
s3_client = boto3.client('s3')
obj = s3_client.get_object(Bucket=bucket, Key=data_key)

records = []
failed_seqs = []

for i, row in enumerate(obj["Body"].iter_lines()):
    parsed_record = parse_record(row)
    if parsed_record:
        records.append(parsed_record)
    
    if i>0 and (i%100_000)==0:
        try:
            save_dataframe(records)
            with open("submissions.csv", encoding="utf-8", errors="replace") as fp:
                cursor.copy_from(fp, "submissions", sep=",", null="", columns=selected_columns)
        except Exception as e:
            print(e)
            print(f"Records {i-100_000}-{i} Failed")
            failed_seqs.append((i-100_000, i))

        records = []
        gc.collect();
        if (i%1_000_000)==0:
            print(f'{i} Records Processed')

if records:
    try:
        save_dataframe(records)
        with open("submissions.csv") as fp:
            cursor.copy_from(fp, "submissions", sep=",", null="", columns=selected_columns)
    except Exception as e:
        print(e)
        #print(f"Records {i-100}-{i} Failed")

need to escape, but no escapechar set
Records 500000-600000 Failed
1000000 Records Processed
need to escape, but no escapechar set
Records 1900000-2000000 Failed
2000000 Records Processed
need to escape, but no escapechar set
Records 2000000-2100000 Failed


In [188]:
#failed_seqs

In [143]:
#!cat submissions.csv

## Logic to Insert Missing Records

In [8]:
s3_client = boto3.client('s3')
obj = s3_client.get_object(Bucket=bucket, Key=data_key)

records = []
failed_seqs = []
start, end, batch_size = 500000, 600000, 1000

for i, row in enumerate(obj["Body"].iter_lines()):
    if i>start and i<=end:
        parsed_record = parse_record(row)
        if parsed_record:
            records.append(parsed_record)

        if i>0 and (i%batch_size)==0:
            try:
                save_dataframe(records)
                with open("submissions.csv", encoding="utf-8", errors="replace") as fp:
                    cursor.copy_from(fp, "submissions", sep=",", null="", columns=selected_columns)
            except Exception as e:
                print(e)
                print(f"Records {i-batch_size}-{i} Failed")
                os.rename('submissions.csv', 'submissions_{}_{}.csv'.format(i-batch_size, i))
                failed_seqs.append((i-batch_size, i))

            records = []
            gc.collect();
            if (i%(batch_size*10))==0:
                print(f'{i} Records Processed')

need to escape, but no escapechar set
Records 549000-550000 Failed
550000 Records Processed
560000 Records Processed
570000 Records Processed
580000 Records Processed
590000 Records Processed
600000 Records Processed


In [9]:
s3_client = boto3.client('s3')
obj = s3_client.get_object(Bucket=bucket, Key=data_key)

records = []
failed_seqs = []
start, end, batch_size = 1900000, 2000000, 1000

for i, row in enumerate(obj["Body"].iter_lines()):
    if i>start and i<=end:
        parsed_record = parse_record(row)
        if parsed_record:
            records.append(parsed_record)

        if i>0 and (i%batch_size)==0:
            try:
                save_dataframe(records)
                with open("submissions.csv", encoding="utf-8", errors="replace") as fp:
                    cursor.copy_from(fp, "submissions", sep=",", null="", columns=selected_columns)
            except Exception as e:
                print(e)
                print(f"Records {i-batch_size}-{i} Failed")
                os.rename('submissions.csv', 'submissions_{}_{}.csv'.format(i-batch_size, i))
                failed_seqs.append((i-batch_size, i))

            records = []
            gc.collect();
            if (i%(batch_size*10))==0:
                print(f'{i} Records Processed')

1910000 Records Processed
1920000 Records Processed
1930000 Records Processed
1940000 Records Processed
1950000 Records Processed
1960000 Records Processed
1970000 Records Processed
1980000 Records Processed
1990000 Records Processed
need to escape, but no escapechar set
Records 1998000-1999000 Failed
2000000 Records Processed


In [10]:
s3_client = boto3.client('s3')
obj = s3_client.get_object(Bucket=bucket, Key=data_key)

records = []
failed_seqs = []
start, end, batch_size = 2000000, 2100000, 1000

for i, row in enumerate(obj["Body"].iter_lines()):
    if i>start and i<=end:
        parsed_record = parse_record(row)
        if parsed_record:
            records.append(parsed_record)

        if i>0 and (i%batch_size)==0:
            try:
                save_dataframe(records)
                with open("submissions.csv", encoding="utf-8", errors="replace") as fp:
                    cursor.copy_from(fp, "submissions", sep=",", null="", columns=selected_columns)
            except Exception as e:
                print(e)
                print(f"Records {i-batch_size}-{i} Failed")
                os.rename('submissions.csv', 'submissions_{}_{}.csv'.format(i-batch_size, i))
                failed_seqs.append((i-batch_size, i))

            records = []
            gc.collect();
            if (i%(batch_size*10))==0:
                print(f'{i} Records Processed')

2010000 Records Processed
2020000 Records Processed
need to escape, but no escapechar set
Records 2022000-2023000 Failed
2030000 Records Processed
2040000 Records Processed
need to escape, but no escapechar set
Records 2049000-2050000 Failed
2050000 Records Processed
2060000 Records Processed
2070000 Records Processed
2080000 Records Processed
2090000 Records Processed
2100000 Records Processed


In [14]:
s3_client = boto3.client('s3')
obj = s3_client.get_object(Bucket=bucket, Key=data_key)

records = []
failed_seqs = []
start, end, batch_size = 2049000, 2050000, 10

for i, row in enumerate(obj["Body"].iter_lines()):
    if i>start and i<=end:
        parsed_record = parse_record(row)
        if parsed_record:
            records.append(parsed_record)

        if i>0 and (i%batch_size)==0:
            try:
                save_dataframe(records)
                with open("submissions.csv", encoding="utf-8", errors="replace") as fp:
                    cursor.copy_from(fp, "submissions", sep=",", null="", columns=selected_columns)
            except Exception as e:
                print(e)
                print(f"Records {i-batch_size}-{i} Failed")
                os.rename('submissions.csv', 'submissions_{}_{}.csv'.format(i-batch_size, i))
                failed_seqs.append((i-batch_size, i))

            records = []
            gc.collect();
            if (i%(batch_size*10))==0:
                print(f'{i} Records Processed')

2049100 Records Processed
2049200 Records Processed
2049300 Records Processed
2049400 Records Processed
2049500 Records Processed
2049600 Records Processed
need to escape, but no escapechar set
Records 2049660-2049670 Failed
need to escape, but no escapechar set
Records 2049670-2049680 Failed
2049700 Records Processed
2049800 Records Processed
2049900 Records Processed
2050000 Records Processed


## Checking Records

In [15]:
import pandas as pd

select_query = "SELECT count(*) FROM submissions"

cursor.execute(select_query)

cursor.fetchall()

[(2218139,)]

## Creating 2nd Submissions Table and Insert Records In It.

In [18]:
query = '''CREATE TABLE submissions2 (  
                                id VARCHAR(16),
                                subreddit_id VARCHAR(24),
                                subreddit text,
                                author VARCHAR(50),
                                created_utc INT,
                                permalink text,
                                title text,
                                selftext text,
                                num_comments INT,
                                score INT)'''


cursor.execute(query)

In [19]:
s3_client = boto3.client('s3')
obj = s3_client.get_object(Bucket=bucket, Key=data_key)

records = []
failed_seqs = []
batch_size = 1000

for i, row in enumerate(obj["Body"].iter_lines()):
    parsed_record = parse_record(row)
    if parsed_record:
        records.append(parsed_record)
    
    if i>0 and (i%batch_size)==0:
        try:
            save_dataframe(records)
            with open("submissions.csv", encoding="utf-8", errors="replace") as fp:
                cursor.copy_from(fp, "submissions2", sep=",", null="", columns=selected_columns)
        except Exception as e:
            print(e)
            print(f"Records {i-batch_size}-{i} Failed")
            failed_seqs.append((i-batch_size, i))

        records = []
        gc.collect();
        if (i%1_000_000)==0:
            print(f'{i} Records Processed')

if records:
    try:
        save_dataframe(records)
        with open("submissions.csv") as fp:
            cursor.copy_from(fp, "submissions2", sep=",", null="", columns=selected_columns)
    except Exception as e:
        print(e)
        #print(f"Records {i-100}-{i} Failed")

need to escape, but no escapechar set
Records 549000-550000 Failed
1000000 Records Processed
need to escape, but no escapechar set
Records 1998000-1999000 Failed
2000000 Records Processed
need to escape, but no escapechar set
Records 2022000-2023000 Failed
need to escape, but no escapechar set
Records 2049000-2050000 Failed


In [24]:
s3_client = boto3.client('s3')
obj = s3_client.get_object(Bucket=bucket, Key=data_key)

records = []
failed_seqs = []
start, end, batch_size = 2049000, 2050000, 10

for i, row in enumerate(obj["Body"].iter_lines()):
    if i>start and i<=end:
        parsed_record = parse_record(row)
        if parsed_record:
            records.append(parsed_record)

        if i>0 and (i%batch_size)==0:
            try:
                save_dataframe(records)
                with open("submissions.csv", encoding="utf-8", errors="replace") as fp:
                    cursor.copy_from(fp, "submissions2", sep=",", null="", columns=selected_columns)
            except Exception as e:
                print(e)
                print(f"Records {i-batch_size}-{i} Failed")
                os.rename('submissions.csv', 'submissions_{}_{}.csv'.format(i-batch_size, i))
                failed_seqs.append((i-batch_size, i))

            records = []
            gc.collect();
            if (i%(batch_size*10))==0:
                print(f'{i} Records Processed')

2049100 Records Processed
2049200 Records Processed
2049300 Records Processed
2049400 Records Processed
2049500 Records Processed
2049600 Records Processed
need to escape, but no escapechar set
Records 2049660-2049670 Failed
need to escape, but no escapechar set
Records 2049670-2049680 Failed
2049700 Records Processed
2049800 Records Processed
2049900 Records Processed
2050000 Records Processed


In [26]:
index_query = "CREATE INDEX submissions2_date ON submissions2(created_utc);"

cursor.execute(index_query)

In [25]:
import pandas as pd

select_query = "SELECT count(*) FROM submissions2"

cursor.execute(select_query)

cursor.fetchall()

[(2218139,)]

## Create Index

In [17]:
index_query = "CREATE INDEX submissions_date ON submissions(created_utc);"

cursor.execute(index_query)

In [247]:
select_query = "SELECT * FROM submissions LIMIT 100;"

cursor.execute(select_query)

rows = cursor.fetchall()

records = []
for row in rows:
    records.append([row[0], row[1],row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[9]])
    
df = pd.DataFrame(records, columns=selected_columns)

print(df.shape)

df.head()

(100, 10)


,id,subreddit_id,subreddit,author,created_utc,permalink,title,selftext,num_comments,score
0,s4jw1,t5_2th52,wallstreetbets,[deleted],1334162440,/r/wallstreetbets/comments/s4jw1/earnings_seas...,Earnings season is here. Place your bets.,I know that /r/investing is a great place for ...,22,13
1,s6r57,t5_2th52,wallstreetbets,[deleted],1334263051,/r/wallstreetbets/comments/s6r57/goog_beat_est...,GOOG - beat estimates; price barely rises.,None,0,2
2,sd5ai,t5_2th52,wallstreetbets,[deleted],1334615377,/r/wallstreetbets/comments/sd5ai/my_poorly_tim...,My poorly timed opening position for AAPL earn...,So I missed out on GOOG; which is probably a g...,21,12
3,se66f,t5_2th52,wallstreetbets,[deleted],1334670090,/r/wallstreetbets/comments/se66f/anyone_bettin...,Anyone betting on VVUS and their potential app...,"""I'm normally a long; but I've created a secon...",0,1
4,sinsk,t5_2th52,wallstreetbets,CheeseYogi,1334877676,/r/wallstreetbets/comments/sinsk/after_hgsi_sp...,After HGSI spikes 97%; will share price drop a...,None,0,4


In [85]:
sorted_cols = sorted(df.columns.values) ### Read first 100 Rows

print("Total Cols : {}\n".format(len(sorted_cols)))
print(sorted_cols)

Total Cols : 35

['approved_by', 'author', 'author_flair_css_class', 'author_flair_text', 'banned_by', 'clicked', 'created', 'created_utc', 'distinguished', 'domain', 'downs', 'edited', 'hidden', 'id', 'is_self', 'likes', 'link_flair_css_class', 'link_flair_text', 'media', 'media_embed', 'name', 'num_comments', 'num_reports', 'over_18', 'permalink', 'saved', 'score', 'selftext', 'selftext_html', 'subreddit', 'subreddit_id', 'thumbnail', 'title', 'ups', 'url']


In [101]:
rows = df.to_dict(orient="records")

rows[1]

{'downs': 2,
 'link_flair_text': '',
 'distinguished': '',
 'media': '',
 'url': 'http://www.reddit.com/r/wallstreetbets/comments/sd5ai/my_poorly_timed_opening_position_for_aapl/',
 'link_flair_css_class': '',
 'id': 'sd5ai',
 'edited': True,
 'num_reports': '',
 'created_utc': 1334615377,
 'banned_by': '',
 'name': 't3_sd5ai',
 'subreddit': 'wallstreetbets',
 'title': 'My poorly timed opening position for AAPL earnings bet',
 'author_flair_text': '',
 'is_self': True,
 'author': '[deleted]',
 'media_embed': {},
 'permalink': '/r/wallstreetbets/comments/sd5ai/my_poorly_timed_opening_position_for_aapl/',
 'author_flair_css_class': '',
 'selftext': "So I missed out on GOOG, which is probably a good thing given their split/dividend announcement really put a damper on things.  But I'm not planning on missing out on AAPL and today was a perfect day to lock in a bet.  Unfortunately for me I got greedy and subsequently got slapped on the wrist.\n\nPosition: Bought an OTM straddle with intenti

## Some Common Attributes Which Can Be Useful

1. id
2. subreddit_id (unique id, it is referred to by comments.json with parent_id)
3. title
4. selftext
5. selftext_html (Same as previous one)
6. num_comments
7. ups
8. downs
9. created_utc (if time needed)
10. score (This one is (ups - down), so it can be avoided i guess)
11. author